# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [61]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xlsxwriter
import math
import yfinance as yf
import pandas_ta
import datetime as dt
import warnings
from scipy.stats import percentileofscore as score

warnings.filterwarnings('ignore') 

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
symbols_list = sp500['Symbol'].unique().tolist()

symbols_list


## Get One-year Price Returns

In [26]:
msft = yf.Ticker("MSFT")
hist = msft.history(period="1y")

# calculate return
prev_price = hist[hist.keys()[0]][0]
curr_price = hist[hist.keys()[0]][-1]
stock_return = (curr_price / prev_price) - 1

stock_return

0.5966853349578014

In [28]:
cols = ['Ticker', 'Price', 'One Year Return', 'Number of Shares to Buy']
res_df = pd.DataFrame(columns = cols)
res_df

,Ticker,Price,One Year Return,Number of Shares to Buy


## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [29]:
res_df = pd.DataFrame(columns = cols)

symbols_string = " ".join(str(element) for element in symbols_list)
tickers = yf.Tickers(symbols_string)

symbols_string

'MMM AOS ABT ABBV ACN ADBE AMD AES AFL A APD ABNB AKAM ALB ARE ALGN ALLE LNT ALL GOOGL GOOG MO AMZN AMCR AEE AAL AEP AXP AIG AMT AWK AMP AME AMGN APH ADI ANSS AON APA AAPL AMAT APTV ACGL ADM ANET AJG AIZ T ATO ADSK ADP AZO AVB AVY AXON BKR BALL BAC BK BBWI BAX BDX BRK-B BBY BIO TECH BIIB BLK BX BA BKNG BWA BXP BSX BMY AVGO BR BRO BF-B BLDR BG CDNS CZR CPT CPB COF CAH KMX CCL CARR CTLT CAT CBOE CBRE CDW CE COR CNC CNP CDAY CF CHRW CRL SCHW CHTR CVX CMG CB CHD CI CINF CTAS CSCO C CFG CLX CME CMS KO CTSH CL CMCSA CMA CAG COP ED STZ CEG COO CPRT GLW CTVA CSGP COST CTRA CCI CSX CMI CVS DHR DRI DVA DE DAL XRAY DVN DXCM FANG DLR DFS DG DLTR D DPZ DOV DOW DHI DTE DUK DD EMN ETN EBAY ECL EIX EW EA ELV LLY EMR ENPH ETR EOG EPAM EQT EFX EQIX EQR ESS EL ETSY EG EVRG ES EXC EXPE EXPD EXR XOM FFIV FDS FICO FAST FRT FDX FIS FITB FSLR FE FI FLT FMC F FTNT FTV FOXA FOX BEN FCX GRMN IT GEHC GEN GNRC GD GE GIS GM GPC GILD GPN GL GS HAL HIG HAS HCA PEAK HSIC HSY HES HPE HLT HOLX HD HON HRL HST HWM HPQ HUB

In [30]:
for symbol in symbols_list:
    hist = tickers.tickers[symbol].history(period="1y")
    prev_price = hist[hist.keys()[0]][0]
    curr_price = hist[hist.keys()[0]][-1]
    stock_return = (curr_price / prev_price) - 1
    res_df.loc[len(res_df)] = [symbol, curr_price, stock_return, 'N/A']

res_df

,Ticker,Price,One Year Return,Number of Shares to Buy
0,MMM,105.879997,-0.070445,N/A
1,AOS,81.949997,0.489011,N/A
2,ABT,108.860001,0.031278,N/A
3,ABBV,153.289993,-0.023915,N/A
4,ACN,350.000000,0.337751,N/A
...,...,...,...,...
498,YUM,129.899994,0.030477,N/A
499,ZBRA,266.769989,0.080653,N/A
500,ZBH,120.300003,-0.039287,N/A
501,ZION,44.279999,-0.036935,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [37]:
res_df.sort_values('One Year Return', ascending = False, inplace = True)
res_df = res_df[:50]
res_df.reset_index(inplace = True)
res_df

,level_0,index,Ticker,Price,One Year Return,Number of Shares to Buy
0,0,348,NVDA,491.950012,2.238802,N/A
1,1,312,META,355.579987,2.064552,N/A
2,2,407,RCL,127.160004,1.572007,N/A
3,3,79,BLDR,164.839996,1.547365,N/A
4,4,455,UBER,61.700001,1.522486,N/A
5,5,88,CCL,19.139999,1.435114,N/A
6,6,387,PHM,103.029999,1.274417,N/A
7,7,6,AMD,140.479996,1.225955,N/A
8,8,361,PANW,300.329987,1.098449,N/A
9,9,75,AVGO,1127.979980,1.097550,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [41]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')

    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number, please try again:')
        portfolio_size = input('Enter the size of your portfolio:')

portfolio_input()
portfolio_size

Enter the size of your portfolio: 1000000


'1000000'

In [45]:
position_size = float(portfolio_size)/len(res_df.index)
position_size

20000.0

In [47]:
for i in range(0, len(res_df.index)):
    res_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/res_df.loc[i, 'Price'])

In [48]:
res_df

,level_0,index,Ticker,Price,One Year Return,Number of Shares to Buy
0,0,348,NVDA,491.950012,2.238802,40
1,1,312,META,355.579987,2.064552,56
2,2,407,RCL,127.160004,1.572007,157
3,3,79,BLDR,164.839996,1.547365,121
4,4,455,UBER,61.700001,1.522486,324
5,5,88,CCL,19.139999,1.435114,1044
6,6,387,PHM,103.029999,1.274417,194
7,7,6,AMD,140.479996,1.225955,142
8,8,361,PANW,300.329987,1.098449,66
9,9,75,AVGO,1127.979980,1.097550,17


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [94]:
hqm_col = ['Ticker', 
           'Price', 
           'Number of Shares to Buy', 
           'One-Year Price Return', 
           'One-Year Return Percentile',
           'Six-Month Price Return', 
           'Six-Month Return Percentile',
           'Three-Month Price Return', 
           'Three-Month Return Percentile',
           'One-Month Price Return', 
           'One-Month Return Percentile',
           'HQM Score'
          ]
hqm_df = pd.DataFrame(columns = hqm_col)
hqm_df

for symbol in symbols_list:
    hist = tickers.tickers[symbol].history(period="1y")
    trading_days = len(hist[hist.keys()[0]])
    y1_price = hist[hist.keys()[0]][0]
    m6_price = hist[hist.keys()[0]][math.floor(trading_days/2)]
    m3_price = hist[hist.keys()[0]][-math.floor(trading_days/4)]
    m1_price = hist[hist.keys()[0]][-math.floor(trading_days/12)]
    curr_price = hist[hist.keys()[0]][-1]
    y1_return = (curr_price / y1_price) - 1
    m6_return = (curr_price / m6_price) - 1
    m3_return = (curr_price / m3_price) - 1
    m1_return = (curr_price / m1_price) - 1
    hqm_df.loc[len(hqm_df)] = [symbol, 
                               curr_price, 
                               'N/A',
                               y1_return, 
                               'N/A',
                               m6_return, 
                               'N/A',
                               m3_return, 
                               'N/A',
                               m1_return, 
                               'N/A',
                               'N/A']
                              

In [95]:
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,105.879997,N/A,-0.070445,N/A,0.092419,N/A,0.142528,N/A,0.102917,N/A,N/A
1,AOS,81.949997,N/A,0.489011,N/A,0.161005,N/A,0.249257,N/A,0.084999,N/A,N/A
2,ABT,108.860001,N/A,0.031278,N/A,0.010080,N/A,0.133271,N/A,0.061014,N/A,N/A
3,ABBV,153.289993,N/A,-0.023914,N/A,0.160593,N/A,0.006399,N/A,0.105191,N/A,N/A
4,ACN,350.000000,N/A,0.337751,N/A,0.187154,N/A,0.125690,N/A,0.048469,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,129.899994,N/A,0.030477,N/A,-0.029057,N/A,0.052451,N/A,0.018424,N/A,N/A
499,ZBRA,266.769989,N/A,0.080653,N/A,-0.025854,N/A,0.190087,N/A,0.171843,N/A,N/A
500,ZBH,120.300003,N/A,-0.039287,N/A,-0.155852,N/A,0.070514,N/A,0.063849,N/A,N/A
501,ZION,44.279999,N/A,-0.036935,N/A,0.671181,N/A,0.341080,N/A,0.291339,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [130]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_df.index:
    for time_period in time_periods:
        return_col = f'{time_period} Return Percentile'
        percentile_col = f'{time_period} Price Return'
        hqm_df.loc[row, return_col] = score(hqm_df[percentile_col], hqm_df.loc[row, percentile_col])/100

hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,CCL,19.139999,1044,1.435114,0.94,0.297627,0.52,0.377970,0.8,0.331015,0.96,96.769384
1,RCL,127.160004,157,1.572007,1.0,0.294645,0.5,0.396289,0.92,0.221049,0.84,96.172962
2,BLDR,164.839996,121,1.547365,0.98,0.282901,0.44,0.336360,0.68,0.232725,0.9,95.178926
3,PHM,103.029999,194,1.274417,0.92,0.353422,0.68,0.392949,0.9,0.171886,0.62,94.930417
4,URI,570.169983,35,0.650069,0.54,0.382729,0.78,0.291424,0.46,0.231788,0.88,94.184891
5,AVGO,1127.979980,17,1.097549,0.86,0.374883,0.74,0.383159,0.84,0.160853,0.5,94.135189
6,EXPE,154.880005,129,0.841617,0.72,0.472383,0.94,0.544784,1.0,0.143195,0.4,94.035785
7,DHI,151.330002,132,0.702286,0.58,0.265245,0.38,0.399570,0.96,0.195340,0.72,93.240557
8,AMD,140.479996,142,1.225955,0.9,0.293912,0.48,0.450940,0.98,0.150815,0.46,92.942346
9,BX,129.740005,154,0.807591,0.66,0.468141,0.92,0.231179,0.2,0.217873,0.82,92.842942


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [131]:
from statistics import mean

for row in hqm_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{time_period} Return Percentile'])
    hqm_df.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,CCL,19.139999,1044,1.435114,0.94,0.297627,0.52,0.377970,0.8,0.331015,0.96,0.805
1,RCL,127.160004,157,1.572007,1.0,0.294645,0.5,0.396289,0.92,0.221049,0.84,0.815
2,BLDR,164.839996,121,1.547365,0.98,0.282901,0.44,0.336360,0.68,0.232725,0.9,0.75
3,PHM,103.029999,194,1.274417,0.92,0.353422,0.68,0.392949,0.9,0.171886,0.62,0.78
4,URI,570.169983,35,0.650069,0.54,0.382729,0.78,0.291424,0.46,0.231788,0.88,0.665
5,AVGO,1127.979980,17,1.097549,0.86,0.374883,0.74,0.383159,0.84,0.160853,0.5,0.735
6,EXPE,154.880005,129,0.841617,0.72,0.472383,0.94,0.544784,1.0,0.143195,0.4,0.765
7,DHI,151.330002,132,0.702286,0.58,0.265245,0.38,0.399570,0.96,0.195340,0.72,0.66
8,AMD,140.479996,142,1.225955,0.9,0.293912,0.48,0.450940,0.98,0.150815,0.46,0.705
9,BX,129.740005,154,0.807591,0.66,0.468141,0.92,0.231179,0.2,0.217873,0.82,0.65


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [132]:
hqm_df.sort_values('HQM Score', ascending = False, inplace = True)
hqm_df = hqm_df[:50]
hqm_df.reset_index(drop = True, inplace = True)
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,RCL,127.160004,157,1.572007,1.0,0.294645,0.5,0.396289,0.92,0.221049,0.84,0.815
1,CCL,19.139999,1044,1.435114,0.94,0.297627,0.52,0.377970,0.8,0.331015,0.96,0.805
2,PHM,103.029999,194,1.274417,0.92,0.353422,0.68,0.392949,0.9,0.171886,0.62,0.78
3,EXPE,154.880005,129,0.841617,0.72,0.472383,0.94,0.544784,1.0,0.143195,0.4,0.765
4,BLDR,164.839996,121,1.547365,0.98,0.282901,0.44,0.336360,0.68,0.232725,0.9,0.75
5,AVGO,1127.979980,17,1.097549,0.86,0.374883,0.74,0.383159,0.84,0.160853,0.5,0.735
6,AMD,140.479996,142,1.225955,0.9,0.293912,0.48,0.450940,0.98,0.150815,0.46,0.705
7,UBER,61.700001,324,1.522486,0.96,0.380622,0.76,0.386205,0.88,0.112313,0.16,0.69
8,URI,570.169983,35,0.650069,0.54,0.382729,0.78,0.291424,0.46,0.231788,0.88,0.665
9,DHI,151.330002,132,0.702286,0.58,0.265245,0.38,0.399570,0.96,0.195340,0.72,0.66


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [135]:
portfolio_input()

Enter the size of your portfolio: 1000000


In [136]:
position_size = float(portfolio_size)/len(res_df.index)
position_size

20000.0

In [137]:
for i in range(0, len(hqm_df.index)):
    hqm_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_df.loc[i, 'Price'])

hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,RCL,127.160004,157,1.572007,1.0,0.294645,0.5,0.396289,0.92,0.221049,0.84,0.815
1,CCL,19.139999,1044,1.435114,0.94,0.297627,0.52,0.377970,0.8,0.331015,0.96,0.805
2,PHM,103.029999,194,1.274417,0.92,0.353422,0.68,0.392949,0.9,0.171886,0.62,0.78
3,EXPE,154.880005,129,0.841617,0.72,0.472383,0.94,0.544784,1.0,0.143195,0.4,0.765
4,BLDR,164.839996,121,1.547365,0.98,0.282901,0.44,0.336360,0.68,0.232725,0.9,0.75
5,AVGO,1127.979980,17,1.097549,0.86,0.374883,0.74,0.383159,0.84,0.160853,0.5,0.735
6,AMD,140.479996,142,1.225955,0.9,0.293912,0.48,0.450940,0.98,0.150815,0.46,0.705
7,UBER,61.700001,324,1.522486,0.96,0.380622,0.76,0.386205,0.88,0.112313,0.16,0.69
8,URI,570.169983,35,0.650069,0.54,0.382729,0.78,0.291424,0.46,0.231788,0.88,0.665
9,DHI,151.330002,132,0.702286,0.58,0.265245,0.38,0.399570,0.96,0.195340,0.72,0.66


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [152]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_df.to_excel(writer, sheet_name = 'Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [153]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [154]:
col_format = {
    'A': ['Ticker', string_template], 
    'B': ['Price', dollar_template], 
    'C': ['Number of Shares to Buy', integer_template], 
    'D': ['One-Year Price Return', percent_template], 
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template], 
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template], 
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template], 
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for col in col_format.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{col}:{col}', 25, col_format[col][1])
    writer.sheets['Momentum Strategy'].write(f'{col}1', col_format[col][0], col_format[col][1])



## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [155]:
writer.close()